- SQL和DataFrame都可以表示数据操作，他们都会被编译成相同的低级代码
- Hive是Hadoop组件中支持SQL的主流大数据处理工具
- Spark的能力
    - SQL分析人员通过Thrift Server或者Spark的SQL接口利用Spark的计算能力
    - API功能强大
    - 允许使用SQL提取数据并将数据转化成DataFrame处理
- Spark SQL的目的是作为一个在线分析处理（OLAP）数据库而存在，而不是在线事务处理（OLTP）
- 这意味着Spark SQL不适合执行对低延迟要求极高的查询

<h4>如何运行Spark SQL查询</h4>

- Spark SQL CLI
    - 在本地模式的命令行中实现基本的SQL查询
    - Spark SQL CLI无法与Thrift JDBC服务端通信
    - ./bin/spark-sql
 - Spark的可编程SQL接口
    - 除了启用服务器之外，还可以通过任何Spark支持语言的API执行SQL
    - 可以通过SparkSession对象上的sql方法来实现，这将返回一个DataFrame
    - 例如，在Python或Scala中，我们可以运行以下内容
    - spark.sql("SELECT 1 + 1").show()

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Python").getOrCreate()

spark.sql("""<br>
    SELECT user_id, department, first_name FROM professors<br>
    WHERE department IN (SELECT name FROM department WHERE created_date >= '2016-01-01')<br>
""")

- 你可以根据需要在SQL和DataFrame之间实现完全的互操作
- 例如你可以创建一个DataFrame，使用SQL操作它，然后再次作为DataFrame进行操作

In [2]:
spark.read.json("./data/flight-data/json/2015-summary.json").createOrReplaceTempView("some_sql_view")

In [3]:
spark.sql("""
    SELECT DEST_COUNTRY_NAME, sum(count)
    FROM some_sql_view GROUP BY DEST_COUNTRY_NAME
""")\
.where("DEST_COUNTRY_NAME like 'S%'").where("`sum(count)` > 10")\
.count()

12

<h4>SparkSQL Thrift JDBC/ODBC服务器</h4>

- Spark提供了一个Java数据库连接（JDBC）接口，通过它或远程程序可以连接到Spark驱动器，以便执行Spark SQL查询
- 要启动JDBC/ODBC服务器，在Spark目录下运行
    - ./sbin/start-thriftserver.sh
- 此脚本支持全部的./bin/spark-submit命令行选项，要查看配置此Thrift服务器的所有可用选项，请运行
    - ./sbin/start-thriftserver.sh --help
- 默认情况下，服务器监听localhost:10000   
- 对于环境变量配置，请用以下方法
    - export HIVE_SERVER2_THRIFT_PORT=\<listening-port>
    - export HIVE_SERVER2_THRIFT_BIND_HOST=\<listening-port>
    - ./sbin/start-thriftserver.gh --master <master-url>
- 对于系统属性，可以参考下面
    - ./sbin/start-thriftserver.sh 
    - --hiveconf hive.server2.thrift.port=\<listening-port>
    - --hiveconf hive.server2.thrift.bind.host=\<listening-port>
- 可以通过以下命令测试此连接
    - ./bin/beeline
    - !connect jdbc:hive2://localhost:100000

<h4>Catalog</h4>

- Spark SQL中最高级别的抽象是Catalog
- Catalog是一个抽象，用于存储用户的元数据以及其他有用的东西
- 如数据库，数据表，函数和视图
- 它在org.apache.spark.SQL.catalog.Catalog中
- 也包含了执行诸如列举表、数据库和函数之类的操作
- 实际上就是spark sql的另一个编程接口

<h4>数据表</h4>

- 逻辑上等同于DataFrame，和DataFrame的核心区别在于
- DataFrame是在编程语言范围内定义的，数据表是在数据库中定义的

<h4>创建表</h4>

- 如果不使用USING指定格式，Spark将默认为Hive SerDe配置，会比Spark的本机序列化慢得多

In [ ]:
CREATE TABLE flights (
    DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count LONG
)
USING JSON OPTIONS (path './data/flight-data/json/2015-summary.json')

- 可以向表中的某些列添加注释

In [ ]:
CREATE TABLE flights_csv (
    DEST_COUNTRY_NAME STRING,
    ORIGIN_COUNTRY_NAME STRING COMMENT "TEST COMMENT",
    count LONG
)
USING csv OPTIONS (header true, path './data/flight-data/csv/2015-summary.csv')

- 也可以从查询结果创建表

CREATE TABLE IF NOT EXISTS flights_from_select USING parquet AS SELECT * FROM flights

- 也可以写出已分区的数据控制数据布局

CREATE TABLE partitioned_flights USING parquet<br>
PARITIONED BY (DEST_COUNTRY_NAME)<br>
AS SELECT DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME, count FROM flights LIMIT 5

<h4>Spark的托管表和外部表</h4>

- 表存储两类重要信息，分别是表中的数据以及关于表的元数据
- 托管表（managed table）：加载后会将表移入默认的Hive仓库位置，删除会连带元数据与数据一起删除
- 非托管表（unmanaged table）：不会移到仓库目录，只会删除元数据

<h4>创建外部表</h4>

In [2]:
CREATE EXTERNAL TABLE hive_flights (
    DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count LONG
)
ROW FORMAT DELIMITIED FIELDS TERMINATED BY ',' 
LOCATION './data/flight-data-hive'

SyntaxError: invalid syntax (<ipython-input-2-6b158c7576b4>, line 1)

In [ ]:
CREATE EXTERNAL TABLE hive_flights_2
ROW FORMAT DELIMITIED FIELDS TERMINATED BY ',' 
LOCATION './data/flight-data-hive' AS SELECT * FROM flights

<h4>插入表</h4>

In [ ]:
INSERRT INTO flights_from_select
    SELECT DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME, count FROM flights LIMIT 20

- 如果只想写入某个分区，可以提供分区方案

In [ ]:
INSERT INTO partitioned_flights
    PARTITION (DEST_COUNTRY_NAME="UNITED STATES")
    SELECT count, ORIGIN_COUNTRY_NAME FROM flights
    WHERE DEST_COUNTRY_NAME='UNITED STATES' LIMIT 12

- 描述表的元数据
    - DESCRIBE TABLE flights_csv
- 查看数据的分区方案
    - SHOW PARTITIONS partitioned_flights

<h4>刷新表的元数据</h4>

- 维护表的元数据确保从最新的数据集读取数据，有两个命令可刷新表的元数据
- REFRESH TABLE用来刷新与表关联的所有缓存项
- 如果之前缓存了该表，则在下次扫描时会惰性缓存它
    - REFRESH table partitioned_flights
- 另一个相关命令是REPAIR TABLE，它刷新在catalog中维护的分区
    - MSCK REPAIR TABLE partitioned_flights

<h4>删除表</h4>

- 不能删除表，只能drop，使用DROP关键字
- 如果DROP托管表，则数据和表定义都将被删除